John Crowe - Quantitative Asset Management HW #2

# Construct the equal-weighted bond market return, value-weighted bond market return, and lagged total bond market capitalization using CRSP Bond data 1. Your output should be from January 1926 to December 2023, at a monthly frequency.

Same as problem set 1, we are going to import the CRSP data for Stocks, Bonds and the Risk Free Rate. We will combine the dlret and ret data for the stocks and create 'cum_div_ret' to encompass all returns for stocks. We will filter our stocks to only use the 10 and 11 sharecodes and the 1, 2 and 3 exchanges, per the French Fama website. We will use the French Fama method for replicating the excess stock returns for the risk parity paper.

In [1]:
import wrds
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import pandas_datareader
import datetime
import numpy as np
from scipy.stats import skew, kurtosis

Below I connected the script to the “WRDS” servers so I could access the data for the stocks, bonds and risk free rate.

In [3]:
id = 'johncrowe'
conn = wrds.Connection(id)

Enter your WRDS username [jcrowe95]:johncrowe
Enter your password:········
WRDS recommends setting up a .pgpass file.
Create .pgpass file now [y/n]?: y
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


Below is code copied from the professor and shown in class. This allows us to connect to the CRSP servers and extract data that we otherwise would not have access to.

In [4]:
crsp_raw = conn.raw_sql("""
                      select a.permno, a.permco, a.date, b.shrcd, b.exchcd,
                      a.ret, a.retx, a.shrout, a.prc, a.cfacshr, a.cfacpr
                      from crspq.msf as a 
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      and b.namedt<=a.date
                      and a.date<=b.nameendt
                      where a.date between '01/01/1900' and '12/31/2023'
                      """)

Below is some data cleaning and rearraning. First we arrange by date and permno, then we manipulate the date portion so that we can ensure they are in the proper order. A lot of this code was provided by the professor.

In [5]:
crsp_raw = crsp_raw.sort_values(['permno','date']).reset_index(drop=True).copy()
crsp_raw[['permno', 'permco']] = crsp_raw[['permno', 'permco']].astype(int)
crsp_raw['date'] = pd.to_datetime(crsp_raw['date'], format='%y-%m-%d') + MonthEnd(0)
crsp_raw['prc'] = np.absolute(crsp_raw['prc'])

In [6]:
dlret_raw = conn.raw_sql("""
                       select permno, dlret, dlstdt, dlstcd
                       from crspq.msedelist
                       """)

Below we are retrieving the delisted data for the stocks so that we can create our 'cum_div_ret' tab which we use for calculating our stock returns.

In [7]:
dlret_raw = dlret_raw.sort_values(['permno', 'dlstdt']).reset_index(drop=True).copy()
dlret_raw.permno = dlret_raw.permno.astype(int)
dlret_raw['dlstdt'] = pd.to_datetime(dlret_raw['dlstdt'])
dlret_raw['date'] = dlret_raw['dlstdt'] + MonthEnd(0)

Here we have all of the relevant stock columns we need for creating our final dataframe.

In [8]:
crsp_data = crsp_raw[['date', 'permno', 'shrcd', 'exchcd', 'ret', 'prc', 'shrout']].copy()
crsp_data = crsp_data.dropna()

Here we merge our stock data frame with the delisted stock returns and fill the 'Na' values with 0, per instruction from the Fama French website. We use an outer merge so that we do not drop any information about the stocks and keep all we have available.

In [9]:
crsp_Stocks = crsp_data.merge(dlret_raw[['permno','date','dlret']], how='outer', on=['permno','date'])
crsp_Stocks['dlret'] = crsp_Stocks['dlret'].fillna(0)
crsp_Stocks['shrcd'] = crsp_Stocks['shrcd'].fillna(method='ffill')
crsp_Stocks['exchcd'] = crsp_Stocks['exchcd'].fillna(method='ffill')
crsp_Stocks['ret'] = crsp_Stocks['ret'].fillna(0)
crsp_Stocks['prc'] = crsp_Stocks['prc'].fillna(0)
crsp_Stocks['shrout'] = crsp_Stocks['shrout'].fillna(0)

/tmp/ipykernel_1389/917011597.py:3: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  crsp_Stocks['shrcd'] = crsp_Stocks['shrcd'].fillna(method='ffill')
/tmp/ipykernel_1389/917011597.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  crsp_Stocks['exchcd'] = crsp_Stocks['exchcd'].fillna(method='ffill')


Same as problem set 1, we will filter our stocks to only use index 10 and 11 for 'shrcd', and exchanges 1, 2, and 3 per the Fama French website

In [10]:
stocks = crsp_Stocks[((crsp_Stocks['shrcd'] == 10) | (crsp_Stocks['shrcd'] == 11)) & ((crsp_Stocks['exchcd'] == 1) | (crsp_Stocks['exchcd'] == 2) | (crsp_Stocks['exchcd'] == 3))]
stocks.loc[:, 'cum_div_ret'] = (1 + stocks['ret']) * (1 + stocks['dlret']) - 1

/tmp/ipykernel_1389/1122760976.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stocks.loc[:, 'cum_div_ret'] = (1 + stocks['ret']) * (1 + stocks['dlret']) - 1


Here we are connecting to the server again to get our risk free data.

In [11]:
rf = conn.raw_sql("""
                      select caldt, t30ret, t90ret
                      from crspq.mcti
                      """)
rf['caldt']   = pd.DataFrame(rf[['caldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
rf = rf.rename(columns={"caldt":"date","t30ret":"rf30","t90ret":"rf90"}).copy()

Here we are connecting to the server to extract our bond data. This is the last step of our data extraction since we now have all the information we need for Stocks, Bonds and Risk Free.

In [12]:
print('Load Bond Data')
bonds = conn.raw_sql("""
                      select kycrspid, mcaldt, tmretnua, tmtotout
                      from crspq.tfz_mth
                      """)
bonds['mcaldt']   = pd.DataFrame(bonds[['mcaldt']].values.astype('datetime64[ns]')) + MonthEnd(0)
bonds = bonds.rename(columns={"mcaldt":"date","tmretnua":"ret","tmtotout":"me","kycrspid":"idCRSP"}).copy()
bonds = bonds.dropna(axis=0, how='any')

Load Bond Data


Using our Bond and Stock data, we want to show the market value in millions. The means a value of 4 would equate to $\$4$ million. We have variable 'me' for bonds and for stocks we have 'prc' and 'shrout'. Per the CRSP manual, 'me' is in millions, 'prc' is in ($), and 'shrout' is in thousands. This means we will have to multiply 'prc' and 'shrout' and divide by 1,000 to get the stock value in millions. The bond value 'me' is already in millions so will remain untouched. We are just calculating returns, so we are not factoring in the risk free rate to determine excess returns. Same as in problem set 1, we iterate through every month and combine the bonds for one month with the bonds of the month before, only keeping bonds present in both of those months. We can then use that to find their lagged market cap and the return on the bonds. The first if statement is for the very first month in the set, since the lagged market value must be calculated using a different method.

In [13]:

def PS2_Q1(df):
    
    years = []
    months = []
    bond_vw_returns = []
    bond_ew_returns = []
    lagged_bond_values = []
    
    # This for loop sorts through all the available unique dates in the data frame in chronological order
    for index, date in enumerate(sorted(df['date'].unique())):
        
        if (index==0):
            # Get bonds for this timestep
            this_bonds = df[df['date'] == date]
            
            # Get Lagged Bond Value of the first time step (divide value by 1 + return)
            lagged_bond_value = (this_bonds['me']/(1+this_bonds['ret'])).sum()
            lagged_bond_values.append(lagged_bond_value)
            
            # Get EW Returns
            bond_ew_return = this_bonds['ret'].mean()
            bond_ew_returns.append(bond_ew_return)
            
            # Get VW Returns
            bond_vw_return = (this_bonds['me'].sum() - lagged_bond_value)/lagged_bond_value
            bond_vw_returns.append(bond_vw_return)
            
        else:
            # Let's merge the bonds that were present in this month and last month's data
            last_bonds = df[df['date'] == last_date]
            this_bonds = df[df['date'] == date]
            bonds_shared = pd.merge(last_bonds[['idCRSP', 'me']], this_bonds[['date', 'idCRSP', 'ret']], how = 'inner', on ='idCRSP')
             
            # Sum last month's bonds to find the lagged market cap
            lagged_bond_value = bonds_shared['me'].sum()
            lagged_bond_values.append(lagged_bond_value)

            # Get VW returns
            bond_vw_return = ((bonds_shared['me']*bonds_shared['ret']).sum())/lagged_bond_value
            bond_vw_returns.append(bond_vw_return)
            
            # Get EW returns
            bond_ew_return = bonds_shared['ret'].mean()
            bond_ew_returns.append(bond_ew_return)

        # Append the month and year, set current date to last_date
        years.append(date.year)
        months.append(date.month)
        last_date = date


    data = {'Year': years,
            'Month': months,
            'Bond_lag_MV' : lagged_bond_values,
            'Bond_Ew_Return': bond_ew_returns,
            'Bond_Vw_Return': bond_vw_returns}

    # Create a DataFrame from the dictionary
    Bond_data = pd.DataFrame(data)

    return(Bond_data)

In [14]:
a1 = PS2_Q1(bonds)
a1

,Year,Month,Bond_lag_MV,Bond_Ew_Return,Bond_Vw_Return
0,1926,1,1.936987e+04,0.005101,0.006821
1,1926,2,1.950200e+04,0.003621,0.003844
2,1926,3,1.888400e+04,0.003812,0.003588
3,1926,4,1.873600e+04,0.003740,0.006393
4,1926,5,1.922700e+04,0.002146,0.002629
...,...,...,...,...,...
1171,2023,8,2.032447e+07,-0.003987,-0.004102
1172,2023,9,2.063664e+07,-0.017542,-0.017521
1173,2023,10,2.079155e+07,-0.011342,-0.011391
1174,2023,11,2.072124e+07,0.033303,0.033461


We can see above the output dataframe comparing the Ew return of the Bond and the Vw return of the bond. There is very minimal difference in these two returns, with value weighting providing marginally better returns than equal weighting through all available data.

# Aggregate stock, bond, and riskless datatables. For each year-month, calculate the lagged market value and excess value-weighted returns for both stocks and bonds. Your output should be from January 1926 to December 2023, at a monthly frequency

In this problem, I borrow many of the methods from PS2_Q1, but I do not call for the function because it was risky using two different dataframes to manipulate data. Therefore I created the bond returns here as well as the stock returns. There were two different risk free return sets available, one for 30 day and one for 90 day risk free. I decided to use the 30 day risk free for two reasons. The first reason was because that is what French, Fama and Asness, Frazzini, and Pedersen used to create these market returns. When I tried using the 90 day risk free, I was not able to reproduce the data correctly. The second reason we use the 30 day risk free data is because we are calculating the monthly returns on the stocks and bonds. Many of these portfolios need to be rebalanced monthly in order to replicate these returns. If I were a portfolio manager, it makes more sense to use the 30 day risk free rate since the portfolio could be changing after 30 days to maintain some of the strategies explored in this paper (like the 60-40 strategy or RP strategy). Therefore, we used the 30 day risk free rate to replicate all of these numbers.

The stock returns were replicated by combining the stocks with the previous months stock using the inner method for merge. We could then use this dataframe to find the lagged market value and value weighted return.

In [15]:
def PS2_Q2(df1, df2, df3):

    years = []
    months = []
    bond_excess_returns = []
    lagged_bond_values = []
    stock_excess_returns = []
    lagged_market_values = []

    for index, date in enumerate(sorted(df1['date'].unique())):
        
        #calculate RF for this timestep
        formatted_date = date.strftime('%Y-%m-%d')
        RF = df3[df3['date'] == formatted_date]['rf30'].iloc[0]
        
        if (index==0):
            
            # Get stocks for this timestep
            this_stocks = df1[df1['date'] == date]
            
            # Get lagged stock value
            lagged_market_value = (this_stocks['prc']*this_stocks['shrout']/(1+this_stocks['cum_div_ret'])).sum()
            lagged_market_values.append(lagged_market_value/1000)
            
            # Get this date stock value
            this_stock_value = (this_stocks['prc']*this_stocks['shrout']).sum()
            
            # Get stock VW returns
            stock_excess_return = (this_stock_value - lagged_market_value)/lagged_market_value - RF
            stock_excess_returns.append(stock_excess_return)
            
            # Get bonds for this timestep
            this_bonds = df2[df2['date'] == date]
            
            # Get lagged bond value
            lagged_bond_value = (this_bonds['me']/(1+this_bonds['ret'])).sum()
            lagged_bond_values.append(lagged_bond_value)
            
            # Get bond VW returns
            bond_excess_return = (this_bonds['me'].sum() - lagged_bond_value)/lagged_bond_value - RF
            bond_excess_returns.append(bond_excess_return)
            
            
        else:
            # Find stocks that existed for both this date and the last date, the other stocks can be ignored
            last_stocks = df1[df1['date'] == last_date]
            this_stocks = df1[df1['date'] == date]
            stocks_shared = pd.merge(last_stocks[['permno', 'prc', 'shrout']], this_stocks[['date', 'permno', 'cum_div_ret']], how = 'inner', on ='permno')

            # Calculate lagged stock market value
            lagged_market_value = (stocks_shared['prc']*stocks_shared['shrout']).sum()
            lagged_market_values.append(lagged_market_value/1000)  
            
            # Calculate stock excess returns
            this_market_value = (stocks_shared['prc']*stocks_shared['shrout']*(1 + stocks_shared['cum_div_ret'])).sum()
            stock_excess_return = ((this_market_value - lagged_market_value)/lagged_market_value) - RF
            stock_excess_returns.append(stock_excess_return)

            # Now let's merge the bonds that were present in this month and last month's data
            last_bonds = df2[df2['date'] == last_date]
            this_bonds = df2[df2['date'] == date]
            bonds_shared = pd.merge(last_bonds[['idCRSP', 'me']], this_bonds[['date', 'idCRSP', 'ret']], how = 'inner', on ='idCRSP')
             
            # Sum last month's bonds to find the lagged market cap
            lagged_bond_value = bonds_shared['me'].sum()
            lagged_bond_values.append(lagged_bond_value)

            # Get returns on bonds this month
            bond_excess_return = (((bonds_shared['me']*bonds_shared['ret']).sum())/lagged_bond_value) - RF
            bond_excess_returns.append(bond_excess_return)

        # Append the month and year, set current date to last_date
        years.append(date.year)
        months.append(date.month)
        last_date = date


    data = {'Year': years,
            'Month': months,
            'Stock_lag_MV': lagged_market_values,
            'Stock_Excess_Vw_Ret' : stock_excess_returns,
            'Bond_lag_MV': lagged_bond_values,
            'Bond_Excess_Vw_Ret': bond_excess_returns}


    # Create a DataFrame from the dictionary
    Stock_Bond_Replication = pd.DataFrame(data)
    return(Stock_Bond_Replication)


In [16]:
a2 = PS2_Q2(stocks, bonds, rf)
a2

,Year,Month,Stock_lag_MV,Stock_Excess_Vw_Ret,Bond_lag_MV,Bond_Excess_Vw_Ret
0,1926,1,2.679810e+04,-0.002787,1.936987e+04,0.003870
1,1926,2,2.678975e+04,-0.036105,1.950200e+04,0.001076
2,1926,3,2.601718e+04,-0.067342,1.888400e+04,0.000810
3,1926,4,2.404646e+04,0.033845,1.873600e+04,0.003321
4,1926,5,2.519361e+04,0.011961,1.922700e+04,0.002287
...,...,...,...,...,...,...
1171,2023,8,4.526181e+07,-0.023871,2.032447e+07,-0.008639
1172,2023,9,4.422278e+07,-0.052416,2.063664e+07,-0.021788
1173,2023,10,4.203208e+07,-0.031745,2.079155e+07,-0.016106
1174,2023,11,4.071960e+07,0.088706,2.072124e+07,0.029037


Here we see the Bond market cap and the Stock market cap are roughly the same but can fluctuate depending on the time period. The current stock market cap is around 2x bigger than the current bond market cap. At other times the stock market had a 4x higher cap and at other times (like during the Great Depression) the bond market cap surpassed the stock market cap.

# Calculate the monthly unlevered and levered risk-parity portfolio returns as defined by Asness, Frazzini, and Pedersen (2012). For the levered risk-parity portfolio, match the value-weighted portfolio’s $\hat{σ}$ over the longest matched holding period of both. Your output should be from January 1926 to December 2023, at a monthly frequency

We already calculated the bond and stock excess return in questions 1 and 2, so we are familiar with that procedure. The excess value weighted return was calculated by investing in stocks and bonds in proportion to their lagged market value, therefore the exact proportion invested in stocks and bond changed every month depending on the cap relative to both markets. The excess 60-40 return invested 60% in stocks and 40% in bonds, regardless of their respective market caps and is rebalanced monthly to maintain weights.

The stock/bond inverse sigma hat was found by taking the inverse of the rolling 3 year monthly volatility. While there are ways to calculate inverse sigma hat for the first 3 years of data (1926-1928), I thought the results would not be reliable enough for our RP portfolio. Therefore, I decided to use 'NaN' values for the risk parity portfolios for the first 3 years. We can also note that since bonds usually have lower volatility than stocks, the inverse sigma hat of the bonds was typically around 10 times higher than for the stocks. This means the RP portfolios tend to invest much more in the bonds.

The Unlevered K was calculated for each month with the equation below, per the equation in Asness, Frazzini, and Pedersen (2012).

$$
K_{unlevered} = \frac{1}{\hat{\sigma}_{bonds} + \hat{\sigma}_{stocks}}
$$

This equation is used so that we can invest in stocks and bonds based on their volatility (the more volatility, the less we invest). Multiplying $K_{unlevered}$ by $\hat{\sigma}_{bonds}$ and $\hat{\sigma}_{stocks}$ gives the portfolio weights for bonds and stocks, respectively. This is how we created our RP unlevered portfolio so that we can target equal risk allocation across the available instruments with it being rebalanced monthly.

For the RP portfolio, these weights are multiplied by a constant to match the ex post realized volatility of the value-weighted benchmark. This means the RP portfolio will have a constant K every month of the portfolio. Below, we find K to be around 0.020556. This was optimized in the for loop below, which goes through many different proposed values of K and finds which K value creates the RP portfolio with the closest volatility to the value weighted portfolio. Once we had this K, we were able to weigh this portfolio using the same method as for the unlevered portfolio.


In [17]:

def PS2_Q3(df1, df2, df3):
    
    df = PS2_Q2(df1, df2, df3)
    
    # Find VW Stock columns, creating total market value column and stock and bond weights
    df['total_market_value'] = df['Stock_lag_MV'] + df['Bond_lag_MV']
    df['v_weight_stocks'] = df['Stock_lag_MV']/df['total_market_value']
    df['v_weight_bonds'] = df['Bond_lag_MV']/df['total_market_value']
    df['Excess_Vw_Ret'] = df['v_weight_stocks']*df['Stock_Excess_Vw_Ret'] + df['v_weight_bonds']*df['Bond_Excess_Vw_Ret']

    # Create 60-40 returns
    df['Excess_60_40_Ret'] = .6*df['Stock_Excess_Vw_Ret'] + .4*df['Bond_Excess_Vw_Ret']

    # Create stock/bond volatility columns
    # Now we will add the volatility for stocks and bonds and shift them forward once because of the lag between
    # returns and bond/market value
    df['Stock_volatility'] = df['Stock_Excess_Vw_Ret'].rolling(window=36).std().shift(1)
    df['Bond_volatility'] = df['Bond_Excess_Vw_Ret'].rolling(window=36).std().shift(1)

    # Use volatility column to create inverse sigma hat columns
    df['Stock_inverse_sigma_hat'] = 1/df['Stock_volatility']
    df['Bond_inverse_sigma_hat'] = 1/df['Bond_volatility']

    # Use inverse sigma hat columns to create unlevered_k column
    df['Unlevered_K'] = 1/(df['Stock_inverse_sigma_hat'] + df['Bond_inverse_sigma_hat'])
    
    # Use unlevered_k column to find portfolio weights for unlevered rp
    df['Excess_Unlevered_RP_Ret'] = df['Unlevered_K']*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret'] + df['Unlevered_K']*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret']

    # Find std for Excess VW for our RP levered portfolio
    Excess_Vw_Ret_std = df['Excess_Vw_Ret'].std()
    
    # Now lets find Levered_K, such that our leveraged portfolio is equal to the VW Volatility
    min_diff = 10000
    
    for k_test in range(1,50001):
        
        k_test = k_test/1000000
        Leveraged_Ret = k_test*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret'] + k_test*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret']
        Leveraged_Ret_std = Leveraged_Ret.std()
    
        diff = abs(Leveraged_Ret_std - Excess_Vw_Ret_std)
    
        if (diff<min_diff):
            min_diff = diff
            k = k_test
            
    # Create our levered_k value
    df['Levered_k'] = k
    
    # Use our levered_k value for our levered portfolio weights
    df['Excess_Levered_RP_Ret'] = k*df['Bond_inverse_sigma_hat']*df['Bond_Excess_Vw_Ret'] + k*df['Stock_inverse_sigma_hat']*df['Stock_Excess_Vw_Ret']

    # Drop the columns we created for theses calculations that we will not need
    df.drop(columns=['Stock_lag_MV', 'Bond_lag_MV', 'total_market_value', 'v_weight_stocks', 'v_weight_bonds', 'Stock_volatility', 'Bond_volatility'], inplace=True)
    
    return(df)

In [18]:
a3 = PS2_Q3(stocks, bonds, rf)
a3

,Year,Month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_inverse_sigma_hat,Bond_inverse_sigma_hat,Unlevered_K,Excess_Unlevered_RP_Ret,Levered_k,Excess_Levered_RP_Ret
0,1926,1,-0.002787,0.003870,0.000006,-0.000124,NaN,NaN,NaN,NaN,0.020556,NaN
1,1926,2,-0.036105,0.001076,-0.020441,-0.021233,NaN,NaN,NaN,NaN,0.020556,NaN
2,1926,3,-0.067342,0.000810,-0.038680,-0.040081,NaN,NaN,NaN,NaN,0.020556,NaN
3,1926,4,0.033845,0.003321,0.020477,0.021635,NaN,NaN,NaN,NaN,0.020556,NaN
4,1926,5,0.011961,0.002287,0.007774,0.008091,NaN,NaN,NaN,NaN,0.020556,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023,8,-0.023871,-0.008639,-0.019151,-0.017778,18.813102,77.917704,0.010338,-0.011602,0.020556,-0.023069
1172,2023,9,-0.052416,-0.021788,-0.042671,-0.040165,19.153556,77.869270,0.010307,-0.027834,0.020556,-0.055513
1173,2023,10,-0.031745,-0.016106,-0.026569,-0.025489,18.988784,76.149721,0.010511,-0.019227,0.020556,-0.037602
1174,2023,11,0.088706,0.029037,0.068582,0.064838,18.919924,75.401337,0.010602,0.041006,0.020556,0.079505


# Replicate and report Panel A of Table 2 in Asness, Frazzini, and Pedersen (2012) Except for Alpha and t-stat of Alpha columns. Specifically, for all strategies considered, report the annualized average excess returns, t-statistic of the average excess returns, annualized volatility, annualized Sharpe Ratio, skewness, and excess kurtosis. Your sample should be from January 1929 to June 2010, at monthly frequency. Match the format of the table to the extent possible. Discuss the difference between your table and the table reported in the paper. It is zero? If not, justify whether the difference is economically negligible or not. What are the reasons for a nonzero difference?

Below you can see the PS2_Q4 function take the dataframe output from PS2_Q3 and calculate the annualized mean, std, etc. using the built in functions on Python. While the replication is not exact, it tends to range from less than 1 basis point off to around 10 basis points off. While the difference is not nonzero, it can be economically negligible in certain situations. One of these could be when the assets under management is not big enough for a few basis points of error to matter. If the portfolio amount was in the millions, this would probably not be a huge issue since it would equate to a couple of hundred dollars for a few basis points of error. However, if the portfolio was large enough, we would need to improve this replication and find exactly why it is not perfectly aligned with the Asness (2012) numbers.

There are several reasons why our replication was not exact. The authors may have employed specific methodologies or adjustments in their calculations that we are not aware of or cannot replicate exactly. For instance, they might have applied certain filters, transformations, or adjustments to the data that are not explicitly stated in the paper. Replicating academic research often involves making assumptions or approximations when exact methodologies are not available or certain details are not provided. These assumptions could differ from those made by the original authors and contribute to differences in results. Our excess returns on stocks were around 4 basis points off, and this leads to all the other portfolios being slightly off due to them using these excess returns numbers.

Lastly, some of the t-stat measurements were more than 10 basis points off. This would follow from above because of dicrepancies in the excess return replication. Therefore the excess return replication would have to be improved first to improve the t-stat.

In [19]:
def PS2_Q4(df):

    Stock_Excess_Vw_mean = 1200*(df['Stock_Excess_Vw_Ret']).mean()
    Stock_Excess_Vw_std = np.sqrt(12)*100*(df['Stock_Excess_Vw_Ret']).std()
    Stock_Excess_Vw_t_stat = np.sqrt(len(df))*(a3['Stock_Excess_Vw_Ret']).mean()/(df['Stock_Excess_Vw_Ret']).std()
    Stock_Excess_Vw_SR = Stock_Excess_Vw_mean/Stock_Excess_Vw_std
    Stock_Excess_Vw_Skew = skew(df['Stock_Excess_Vw_Ret'])
    Stock_Excess_Vw_Kurtosis = kurtosis(df['Stock_Excess_Vw_Ret'])

    Bond_Excess_Vw_mean = 1200*(df['Bond_Excess_Vw_Ret']).mean()
    Bond_Excess_Vw_std = np.sqrt(12)*100*(df['Bond_Excess_Vw_Ret']).std()
    Bond_Excess_Vw_t_stat = np.sqrt(len(df))*(a3['Bond_Excess_Vw_Ret']).mean()/(df['Bond_Excess_Vw_Ret']).std()
    Bond_Excess_Vw_SR = Bond_Excess_Vw_mean/Bond_Excess_Vw_std
    Bond_Excess_Vw_Skew = skew(df['Bond_Excess_Vw_Ret'])
    Bond_Excess_Vw_Kurtosis = kurtosis(df['Bond_Excess_Vw_Ret'])
    
    Excess_Vw_mean = 1200*(df['Excess_Vw_Ret']).mean()
    Excess_Vw_std = np.sqrt(12)*100*(df['Excess_Vw_Ret']).std()
    Excess_Vw_t_stat = np.sqrt(len(df))*(df['Excess_Vw_Ret']).mean()/(df['Excess_Vw_Ret']).std()
    Excess_Vw_SR = Excess_Vw_mean/Excess_Vw_std
    Excess_Vw_Skew = skew(df['Excess_Vw_Ret'])
    Excess_Vw_Kurtosis = kurtosis(df['Excess_Vw_Ret'])

    Excess_60_40_mean = 1200*(df['Excess_60_40_Ret']).mean()
    Excess_60_40_std = np.sqrt(12)*100*(df['Excess_60_40_Ret']).std()
    Excess_60_40_t_stat = np.sqrt(len(df))*(df['Excess_60_40_Ret']).mean()/(df['Excess_60_40_Ret']).std()
    Excess_60_40_SR = Excess_60_40_mean/Excess_60_40_std
    Excess_60_40_Skew = skew(df['Excess_60_40_Ret'])
    Excess_60_40_Kurtosis = kurtosis(df['Excess_60_40_Ret'])

    Excess_Unlevered_RP_mean = 1200*(df['Excess_Unlevered_RP_Ret']).mean()
    Excess_Unlevered_RP_std = np.sqrt(12)*100*(df['Excess_Unlevered_RP_Ret']).std()
    Excess_Unlevered_RP_t_stat = np.sqrt(len(df))*(df['Excess_Unlevered_RP_Ret']).mean()/(df['Excess_Unlevered_RP_Ret']).std()
    Excess_Unlevered_RP_SR = Excess_Unlevered_RP_mean/Excess_Unlevered_RP_std
    Excess_Unlevered_RP_Skew = skew(df['Excess_Unlevered_RP_Ret'])
    Excess_Unlevered_RP_Kurtosis = kurtosis(df['Excess_Unlevered_RP_Ret'])

    Excess_Levered_RP_mean = 1200*(df['Excess_Levered_RP_Ret']).mean()
    Excess_Levered_RP_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret']).std()
    Excess_Levered_RP_t_stat = np.sqrt(len(df))*(a3['Excess_Levered_RP_Ret']).mean()/(df['Excess_Levered_RP_Ret']).std()
    Excess_Levered_RP_SR = Excess_Levered_RP_mean/Excess_Levered_RP_std
    Excess_Levered_RP_Skew = skew(df['Excess_Levered_RP_Ret'])
    Excess_Levered_RP_Kurtosis = kurtosis(df['Excess_Levered_RP_Ret'])

    RP_minus_Vw_mean = 1200*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).mean()
    RP_minus_Vw_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).std()
    RP_minus_Vw_t_stat = np.sqrt(len(df))*(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).mean()/(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret']).std()
    RP_minus_Vw_SR = RP_minus_Vw_mean/RP_minus_Vw_std
    RP_minus_Vw_Skew = skew(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret'])
    RP_minus_Vw_Kurtosis = kurtosis(df['Excess_Levered_RP_Ret'] - df['Excess_Vw_Ret'])

    RP_minus_60_40_mean = 1200*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).mean()
    RP_minus_60_40_std = np.sqrt(12)*100*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).std()
    RP_minus_60_40_t_stat = np.sqrt(len(df))*(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).mean()/(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret']).std()
    RP_minus_60_40_SR = RP_minus_60_40_mean/RP_minus_60_40_std
    RP_minus_60_40_Skew = skew(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret'])
    RP_minus_60_40_Kurtosis = kurtosis(df['Excess_Levered_RP_Ret'] - df['Excess_60_40_Ret'])

    data = {
        'Index': ['CRSP stocks', 'CRSP bonds', 'Value-weighted portfolio', '60/40 portfolio', 'RP, unlevered', 'RP', 'RP minus value-weighted', 'RP minus 60/40'],
        'Excess Return': [Stock_Excess_Vw_mean, Bond_Excess_Vw_mean, Excess_Vw_mean, Excess_60_40_mean, Excess_Unlevered_RP_mean, Excess_Levered_RP_mean, RP_minus_Vw_mean, RP_minus_60_40_mean],
        't-stat': [Stock_Excess_Vw_t_stat, Bond_Excess_Vw_t_stat, Excess_Vw_t_stat, Excess_60_40_t_stat, Excess_Unlevered_RP_t_stat, Excess_Levered_RP_t_stat, RP_minus_Vw_t_stat, RP_minus_60_40_t_stat],
        'Volatility': [Stock_Excess_Vw_std, Bond_Excess_Vw_std, Excess_Vw_std, Excess_60_40_std, Excess_Unlevered_RP_std, Excess_Levered_RP_std, RP_minus_Vw_std, RP_minus_60_40_std],
        'Sharpe Ratio': [Stock_Excess_Vw_SR, Bond_Excess_Vw_SR, Excess_Vw_SR, Excess_60_40_SR, Excess_Unlevered_RP_SR, Excess_Levered_RP_SR, RP_minus_Vw_SR, RP_minus_60_40_SR],
        'Skewness': [Stock_Excess_Vw_Skew, Bond_Excess_Vw_Skew, Excess_Vw_Skew, Excess_60_40_Skew, Excess_Unlevered_RP_Skew, Excess_Levered_RP_Skew, RP_minus_Vw_Skew, RP_minus_60_40_Skew],
        'Kurtosis': [Stock_Excess_Vw_Kurtosis, Bond_Excess_Vw_Kurtosis, Excess_Vw_Kurtosis, Excess_60_40_Kurtosis, Excess_Unlevered_RP_Kurtosis, Excess_Levered_RP_Kurtosis, RP_minus_Vw_Kurtosis, RP_minus_60_40_Kurtosis]
    }

    df3 = pd.DataFrame(data)


    return(df3)




In [28]:
# For the specified dates, we need the index from [36:1014] for January 1929 to June 2010
a3[36:1014]

,Year,Month,Stock_Excess_Vw_Ret,Bond_Excess_Vw_Ret,Excess_Vw_Ret,Excess_60_40_Ret,Stock_inverse_sigma_hat,Bond_inverse_sigma_hat,Unlevered_K,Excess_Unlevered_RP_Ret,Levered_k,Excess_Levered_RP_Ret
36,1929,1,0.047413,-0.004957,0.036297,0.026465,25.507937,329.390033,0.002818,-0.001193,0.020556,-0.008700
37,1929,2,-0.003453,-0.004926,-0.003765,-0.004042,25.416316,324.341709,0.002859,-0.004819,0.020556,-0.034645
38,1929,3,-0.008216,-0.007900,-0.008150,-0.008089,26.041746,314.568248,0.002936,-0.007924,0.020556,-0.055483
39,1929,4,0.014378,0.009008,0.013270,0.012230,27.990249,292.588705,0.003119,0.009477,0.020556,0.062453
40,1929,5,-0.062352,-0.010162,-0.051598,-0.041476,28.023141,268.977638,0.003367,-0.015086,0.020556,-0.092104
...,...,...,...,...,...,...,...,...,...,...,...,...
1009,2010,2,0.033917,0.001110,0.022915,0.020794,17.160692,96.081438,0.008831,0.006081,0.020556,0.014156
1010,2010,3,0.063103,-0.004902,0.040543,0.035901,17.069371,96.337222,0.008818,0.005334,0.020556,0.012435
1011,2010,4,0.019992,0.007587,0.015981,0.015030,16.772107,95.960107,0.008871,0.009433,0.020556,0.021859
1012,2010,5,-0.078919,0.011817,-0.049312,-0.042625,16.833516,95.698473,0.008886,-0.001756,0.020556,-0.004062


In [29]:
# For the specified dates, we need the index from [36:1014] for January 1929 to June 2010
a4 = PS2_Q4(a3[36:1014])
a4

,Index,Excess Return,t-stat,Volatility,Sharpe Ratio,Skewness,Kurtosis
0,CRSP stocks,6.756157,3.841110,19.093289,0.353850,0.228501,7.672394
1,CRSP bonds,1.403863,3.909564,2.932637,0.478703,0.225334,4.161203
2,Value-weighted portfolio,3.534831,2.653318,12.027026,0.293907,-0.536333,4.507039
3,60/40 portfolio,4.615240,3.575412,11.653248,0.396048,0.239134,7.409348
4,"RP, unlevered",2.100865,5.088610,3.727157,0.563664,0.088449,2.621295
5,RP,6.680801,5.204211,12.007800,0.556372,-0.396516,1.965822
6,RP minus value-weighted,3.145969,3.072866,9.242504,0.340381,0.066914,3.585890
7,RP minus 60/40,2.065561,2.114350,8.819418,0.234206,-0.709692,6.449395
